In [2]:
import pandas as pd
pd.set_option('display.width', 120)

#### 5-1) tips 데이터셋을 이용한다.
* tip/total_bill 비율이 0.15를 초과하고, size가 3이상인 경우만 필터링한다.
* 이 조건을 만족하는 손님의 총 인원 수(size의 합)을 구하시오.

In [7]:
filename="https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/tips.csv"
df = pd.read_csv(filename)
# print(df.head())

# tip / total_bill 비율이 0.15를 초과하고, size가 3 이상인 경우만 필터링한다.
df['ratio'] = df['tip'] / df['total_bill']
cond = (df['ratio'] > 0.15) & (df['size'] >= 3)

# 이 조건을 만족하는 손님의 총 인원 수(size의 합)를 구하시오.
result = df[cond]['size'].sum()
# print(result) # 143

#### 5-2) 데이터에서 month가 'Jan'부터 'Jun'까지인 경우를 상반기, 'Jul'부터 'Dec'까지인 경우를 하반기로 나누어 분석한다.
* 상반기와 하반기 각각에 대해 연도별 총 승객 수(passengers)를 계산한다.
* 각 데이터에서, 연도별 승객 수 증가량을 계산하라. 단, 첫 번째 연도는 증가량을 0으로 둔다.
* 상반기와 하반기 각각의 증가량 중, 가장 많이 증가한 연도를 구한다.
* flights 전체 데이터에서, 최대 승객 수를 구한다.
* (상반기 최대 증가 연도) + (하반기 최대 증가 연도) + (최대 승객 수)의 결과를 정수로 출력하시오.

In [31]:
filename = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/flights.csv"
df = pd.read_csv(filename)
# print(df.head())

# 상반기(df1), 하반기(df2) 데이터 분류
# print(df['month'].unique())
df1 = df[df['month'].str.contains('Jan|Feb|Mar|Apr|May|Jun')].copy() 
df2 = df[df['month'].str.contains('Jul|Aug|Sep|Oct|Nov|Dec')].copy()

# 각 데이터에서 연도별 승객 수 증가량을 계산하라. 단, 첫 번째 연도는 증가량을 0으로 둔다.
df1_diff = df1.groupby('year')['passengers'].sum().reset_index()
df1_diff['diff'] = df1_diff['passengers'].diff().fillna(0)
# print(df1_diff.head())
df2_diff = df2.groupby('year')['passengers'].sum().reset_index()
df2_diff['diff'] = df2_diff['passengers'].diff().fillna(0)
# print(df2_diff.head())

# 상반기와 하반기 각각의 증가량 중, 가장 많이 증가한 연도를 구한다.
df1_max_year = int(df1_diff.iloc[df1_diff['diff'].idxmax()]['year'])
df2_max_year = int(df2_diff.iloc[df2_diff['diff'].idxmax()]['year'])
# print(df1_max_year, df2_max_year) # 1960 1959

# flights 전체 데이터에서, 최대 승객 수
max_passengers = df['passengers'].max()
# print(max_passengers) # 622

# (상반기 최대 증가 연도) + (하반기 최대 증가 연도) + (최대 승객 수)의 결과를 정수로 출력
result = df1_max_year + df2_max_year + max_passengers
# print(int(result)) # 4541


#### 5-3) tips 데이터셋에서 다음 조건을 만족하는 손님의 total_bill 평균을 반올림하여 소수점 아래 3자리까지 구하시오.
* total_bill이 자신이 속한 요일(day)의 평균 이상인 손님만 선택한다.
* 위 조건을 만족하는 손님 중 sex가 Female이고, tip이 중앙값 이상인 데이터만 선택한다.

In [ ]:
filename = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/tips.csv"
df = pd.read_csv(filename)
# print(df.head())

# total_bill이 자신이 속한 요일(day)의 평균 이상인 손님만 선택
day_mean = df.groupby('day')['total_bill'].transform('mean')
df2 = df[df['total_bill'] >= day_mean]

# 위 조건을 만족하는 손님 중 sex가 Female이고, tip이 중앙값 이상인 데이터만 선택
df3 = df2[(df2['sex'] == 'Female') & (df2['tip'] >= df2['tip'].median())]

# 다음 조건을 만족하는 손님의 total_bill 평균을 반올림하여 소수점 아래 3자리까지 구하시오
result = df3['total_bill'].mean()
# print(round(result, 3)) # 29.335

29.335


#### 5-4) titanic_dataq.csv를 사용하여 다음 조건을 만족하는 승객 수를 정수로 구하시오.
* Cabin컬럼의 결측치를 Cabin 컬럼의 첫글자 중 최빈값으로 채우기 한 뒤, Cabin컬럼의 값을 첫글자로 변경한다.
* Cabin컬럼의 최빈값에 해당하는 데이터만 추출하여, 나이(Age) 컬럼의 결측치를 Embarked와 Pclass 조합별 평균나이로 채운다.
* Age 컬럼의 결측치로 채운 후, Age가 Embarked와 Pclass 조합별 중앙값 이상이고, Fare가 중앙값보다 높은 승객만 남긴다.

In [ ]:
filename = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/titanic_dataq.csv"
df = pd.read_csv(filename)
# print(df.head())

# Cabin 컬럼의 결측치를 Cabin 컬럼의 첫 글자 중 최빈값으로 채우기 한 뒤, Cabin 컬럼의 값을 첫 글자로 변경한다.
# (예) C12 => C
df['first_words'] = df['Cabin'].astype(str).str[0]
mode_first = df.loc[df['Cabin'].notnull(), 'first_words'].mode()[0]
df['Cabin'] = df['Cabin'].fillna(mode_first)
df['Cabin'] = df['Cabin'].str[0]
# print(df.head())

# Cabin 컬럼의 최빈값에 해당하는 데이터만 추출하여, 나이(Age) 컬럼의 결측치를 Embarked와 Pclass 조합별 평균 나이로 채운다.
df2 = df[df['Cabin'] == mode_first].copy()
df2['Age'] = df2['Age'].fillna(df2.groupby(['Embarked','Pclass'])['Age'].transform('mean'))
# print(df2.isna().sum().to_frame().T)

# Age 컬럼의 결측치를 채운 후, Age가 Embarked와 Pclass 조합별 중앙값 이상이고, Fare가 중앙값보다 높은 승객만 남긴다.
cond1 = df2.groupby(['Embarked', 'Pclass'])['Age'].transform('median')
cond2 = df2.groupby(['Embarked', 'Pclass'])['Fare'].transform('median')
result = len(df2[(df2['Age'] >= cond1) & (df2['Fare'] > cond2)])
# print(int(result)) # 181

#### 5-5) California housing 데이터셋을 활용하여 다음 조건을 모두 만족하는 계산을 수행하시오.

* Latitude 값을 기준으로 데이터를 5개 구간으로 분위수로 분할하여 ocean_proximity 컬럼을 생성한다.
(pd.qcut(..., q=5, labels=False)를 사용할 것)
* 다음의 필터링을 수행하기 전, 후의 데이터에 대해 ocean_proximity별 HouseAge의 중앙값(median)을 계산한다.
* 필터링 전 데이터에 대한 계산 결과를 S1, 필터링 후 데이터에 대한 계산결과를 S2로 저장한다.
* HouseAge가 자신이 속한 ocena_proximity 그룹의 중앙값보다 큰 경우만 필터링한다.
* 최종적으로 S2에서 S1의 값을 뺀(S2 - S1) 값 중 최소값을 정수로 구하시오.

In [ ]:
filename = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/california.csv"
df = pd.read_csv(filename)
# print(df.head())

# Latitude 값을 기준으로 데이터를 5개 구간으로 분위수 분할하여 ocean_proximity 컬럼을 생성한다.
# (pd.qcut(..., q=5, labels=False)를 사용할 것)
df['ocean_proximity'] = pd.qcut(df['Latitude'], q=5, labels=False)
# print(df.head())

# 다음의 필터링을 수행하기 전, 후의 데이터에 대해 ocean_proximity별 HouseAge의 중앙값(median)을 계산한다.
# 필터링 전 데이터에 대한 계산 결과를 S1, 필터링 후 데이터에 대한 계산 결과를 S2로 저장한다.
# 필터링 전
S1 = df.groupby('ocean_proximity')['HouseAge'].transform('median')

# 필터링 후
df2 = df[df['HouseAge'] > S1]
S2 = df2.groupby('ocean_proximity')['HouseAge'].transform('median')

# S2에서 S1을 뺀 값 중 최소값을 정수로 구하시오.
result = int((S2 - S1).min())
print(result) # 7

7
